# Tune Batch Size and Number of Epochs


In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


import tensorflow as tf
import keras

num_classes = 25

In [5]:
def create_model(optimizer='Adam'):
  input = tf.keras.Input(shape=(64,64,3))

  # Block 1
  x = tf.keras.layers.Conv2D(128, kernel_size=(3 ,3), activation='relu', strides=(2, 2), padding="same")(input)
  x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)

  x = tf.keras.layers.Conv2D(64, kernel_size=(4 ,4), activation='relu', padding="valid")(x)
  x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)

  x = tf.keras.layers.Dropout(.3)(x)
  x = tf.keras.layers.Flatten()(x)

  x = tf.keras.layers.Dense(50, activation='relu')(x)
  output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

  # bind all
  model = tf.keras.Model(input, output)

  model.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer = optimizer,
    metrics=["accuracy"]
  )
  return model

In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
X = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/imgs.npy')
Y = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/labels.npy')

In [6]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.656527 using {'optimizer': 'Adam'}
0.317003 (0.001633) with: {'optimizer': 'SGD'}
0.575942 (0.110935) with: {'optimizer': 'RMSprop'}
0.552460 (0.040318) with: {'optimizer': 'Adagrad'}
0.097022 (0.047544) with: {'optimizer': 'Adadelta'}
0.656527 (0.027898) with: {'optimizer': 'Adam'}
0.609670 (0.036918) with: {'optimizer': 'Adamax'}
0.534102 (0.162259) with: {'optimizer': 'Nadam'}
